# Data Preprocessing

## Dropping useless Columns

In [397]:
import pandas as pd 
import numpy as np
import warnings
warnings.filterwarnings("ignore")

In [398]:
df = pd.read_csv("raw_data_facebook_scraper.csv")
df

,id,primary_listing_photo,if_gk_just_listed_tag_on_search_feed,listing_price,strikethrough_price,comparable_price,comparable_price_type,location,is_hidden,is_live,...,legal_disclosure_impressum_url,commerce_badges_info,listing_address,seller_phone_number,vehicle_website_link,dealership_name,seller,energy_efficiency_class_eu,listing_photos,pre_recorded_videos
0,842325545077316,{'image': {'uri': 'https://scontent.fsyd4-2.fn...,NaN,"{'formatted_amount': 'AU$11,300', 'amount_with...",NaN,NaN,NaN,"{'reverse_geocode': {'city': 'Brisbane', 'stat...",False,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1282138740755042,{'image': {'uri': 'https://scontent.fsyd4-2.fn...,NaN,"{'formatted_amount': 'AU$5,000', 'amount_with_...","{'formatted_amount': 'AU$7,000', 'amount': '70...",NaN,NaN,"{'reverse_geocode': {'city': 'Brisbane', 'stat...",False,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,4343086952677152,{'image': {'uri': 'https://scontent.fsyd4-1.fn...,NaN,"{'formatted_amount': 'AU$13,500', 'amount_with...",NaN,NaN,NaN,"{'reverse_geocode': {'city': 'Brisbane', 'stat...",False,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,856599473632809,{'image': {'uri': 'https://scontent.fsyd4-1.fn...,NaN,"{'formatted_amount': 'AU$89,999', 'amount_with...",NaN,NaN,NaN,"{'reverse_geocode': {'city': 'Brisbane', 'stat...",False,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1929741430912801,{'image': {'uri': 'https://scontent.fsyd4-1.fn...,NaN,"{'formatted_amount': 'AU$18,800', 'amount_with...",NaN,NaN,NaN,"{'reverse_geocode': {'city': 'Brisbane', 'stat...",False,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11950,1626734915162026,{'image': {'uri': 'https://scontent-syd2-1.xx....,NaN,"{'amount_with_offset': '590000', 'currency': '...",NaN,NaN,NaN,"{'latitude': -34.928283691406, 'longitude': 13...",False,True,...,NaN,"{'source_summary': None, 'badges': []}",NaN,NaN,NaN,NaN,NaN,NaN,[{'accessibility_caption': 'May be an image of...,[]
11951,2186106862127606,{'image': {'uri': 'https://scontent.fbne6-1.fn...,NaN,"{'amount_with_offset': '2495000', 'currency': ...",NaN,NaN,NaN,"{'latitude': -37.768249511719, 'longitude': 14...",False,True,...,NaN,{'source_summary': 'Highly rated on Marketplac...,NaN,NaN,NaN,NaN,NaN,NaN,[{'accessibility_caption': 'May be an image of...,[]
11952,1436459311236585,{'image': {'uri': 'https://scontent.fbne5-1.fn...,NaN,"{'amount_with_offset': '899000', 'currency': '...",NaN,NaN,NaN,"{'latitude': -37.817687988281, 'longitude': 14...",False,True,...,NaN,"{'source_summary': None, 'badges': []}",NaN,NaN,NaN,NaN,NaN,NaN,[{'accessibility_caption': 'No photo descripti...,[]
11953,1084441146768423,{'image': {'uri': 'https://scontent.fbne5-1.fn...,NaN,"{'amount_with_offset': '1000', 'currency': 'AU...",NaN,NaN,NaN,"{'latitude': -38.075866699219, 'longitude': 14...",False,True,...,NaN,"{'source_summary': None, 'badges': []}",NaN,NaN,NaN,NaN,NaN,NaN,[{'accessibility_caption': 'No photo descripti...,[]


In [399]:
df.columns

Index(['id', 'primary_listing_photo', 'if_gk_just_listed_tag_on_search_feed',
       'listing_price', 'strikethrough_price', 'comparable_price',
       'comparable_price_type', 'location', 'is_hidden', 'is_live',
       ...
       'legal_disclosure_impressum_url', 'commerce_badges_info',
       'listing_address', 'seller_phone_number', 'vehicle_website_link',
       'dealership_name', 'seller', 'energy_efficiency_class_eu',
       'listing_photos', 'pre_recorded_videos'],
      dtype='object', length=119)

In [400]:
#checking cols that have all null values in it
all_null_cols_mask = df.isnull().all()
cols_with_all_nulls = df.columns[all_null_cols_mask].tolist()
col = ["id","primary_listing_photo"]
cols = cols_with_all_nulls + col
df.drop(cols,axis=1,inplace=True)

## Removing Null-Only Columns

- To improve data quality, we dropped the columns that had **100% missing values** (all `NaN`).  
- These columns do not contribute any useful information to the model and only increase dataset size and complexity.


In [401]:
# Checking null values columns having 8000 null values 
null_counts = df.isnull().sum()
columns_with_many_nulls = null_counts[null_counts > 8000].index.tolist()
df.drop(columns_with_many_nulls,axis=1,inplace=True)

## Removing Columns with High Missing Values

- Columns containing more than **8000 missing values** were removed.  
- Given the dataset size of **11,955 rows**, such columns had a very high percentage of missing data and were not useful for analysis or modeling.


In [402]:
df.isnull().sum().sum()

np.int64(18118)

In [403]:
columns_to_drop = []
for col in df.columns:
    counts = df[col].value_counts()
    if counts.max() > 11000:
            columns_to_drop.append(col)
columns_to_drop = [col for col in columns_to_drop if col not in ["condition"]]

In [404]:
no_use_cols = ["listing_photos","commerce_badges_info",
              "custom_title","listingUrl","inputUrl",
               "if_viewer_is_buyer","vehicle_features"]
finl_no_use_cols = no_use_cols + columns_to_drop
df.drop(finl_no_use_cols,axis=1,inplace=True)

dropping listing photos, listing url , input url as they are link dropping custom title it work done by listing title dropping if viewer is buyer
no need of dropped vehicle_features becoz it contains only empty [] no use of commerce badges info 

## Dropping Unnecessary Columns

- **Low-variance columns:** We dropped columns where the most repeated value appeared in **more than 11,000 rows (out of 11,955)**, since such columns are almost constant and do not help prediction.  
  ✅ However, we kept 1 important columns: **condition**.

- **Irrelevant link/media columns:** Dropped `listing_photos`, `listing_url`, and `input_url` as they are only URLs/media references.

- **Duplicate information:** Dropped `custom_title` because the same work is already covered by `listing_title`.

- **Not useful for modeling:** Dropped `viewer_is_buyer` and `commerce_badges_info`.

- **Empty feature column:** Dropped `vehicle_features` because it mostly contained empty lists (`[]`) and provided no useful information.


In [405]:
df.columns

Index(['listing_price', 'location', 'marketplace_listing_title',
       'custom_sub_titles_with_rendering_flags', 'redacted_description',
       'creation_time', 'location_text', 'location_vanity_or_id',
       'formatted_price', 'condition', 'logging_id', 'product_item',
       'primary_mp_ent', 'reportable_ent_id', 'hidden_from_friends',
       'share_uri', 'cross_post_info', 'vehicle_exterior_color',
       'vehicle_fuel_type', 'vehicle_interior_color',
       'vehicle_make_display_name', 'vehicle_model_display_name',
       'vehicle_number_of_owners', 'vehicle_odometer_data',
       'vehicle_specifications', 'vehicle_transmission_type'],
      dtype='object')

In [406]:
cols = ["location","location_vanity_or_id","product_item",
        "cross_post_info","hidden_from_friends","reportable_ent_id",
        "primary_mp_ent","logging_id","share_uri",
        "custom_sub_titles_with_rendering_flags","vehicle_number_of_owners"]

In [407]:
df.drop(cols,axis=1,inplace=True)

## More Columns Dropped (Reason-wise)

- **Duplicate location info:** Dropped `location` because the same information is already available in `location_text`.

- **Unnecessary location identifier:** Dropped `location_vanity_or_id` because it mainly contains an ID/number and city name, which we can directly extract from `location_text`.

- **Mostly empty nested data:** Dropped `product_item` since it is a dictionary column where most keys contain `None` values.

- **Not relevant for price prediction:**
  - Dropped `cross_post_info` because it only indicates whether the product is sold or not (not needed for predicting price).
  - Dropped `hidden_from_friends` because it only shows visibility settings (public vs friends), which does not affect price.

- **Unique and system-generated IDs:**  
  Dropped `reportable_ent_id`, `primary_mp_ent`, and `logging_id` because they are system-generated unique identifiers with no predictive value for ML.

- **URL-only column:** Dropped `share_uri` because it only contains the product URL and does not help in price prediction.

- **Duplicate information:** Dropped `custom_sub_titles_with_rendering_flags` because the same information is already present in `vehicle_odometer_data`.

- **High missing values:** Dropped `vehicle_number_of_owners` because it contains more than **6000 missing values** (over half of the dataset), making it unreliable and less useful for modeling.


In [408]:
df

,listing_price,marketplace_listing_title,redacted_description,creation_time,location_text,formatted_price,condition,vehicle_exterior_color,vehicle_fuel_type,vehicle_interior_color,vehicle_make_display_name,vehicle_model_display_name,vehicle_odometer_data,vehicle_specifications,vehicle_transmission_type
0,"{'formatted_amount': 'AU$11,300', 'amount_with...",2015 Hyundai i30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"{'formatted_amount': 'AU$5,000', 'amount_with_...",2020 MG MG3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"{'formatted_amount': 'AU$13,500', 'amount_with...",2016 Hyundai Tucson,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,"{'formatted_amount': 'AU$89,999', 'amount_with...",2008 Nissan GT-R,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,"{'formatted_amount': 'AU$18,800', 'amount_with...",2014 Toyota 86,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11950,"{'amount_with_offset': '590000', 'currency': '...",2002 Mercedes-Benz Sedan,{'text': 'Mercedes Benz kompressor classic sed...,1.767777e+09,"{'text': 'Adelaide, SA'}","{'text': 'AU$5,900'}",USED,white,GASOLINE,off_white,Mercedes-Benz,sedan,"{'unit': 'KILOMETERS', 'value': 181000}","{'co2_emissions': None, 'engine_size': None, '...",AUTOMATIC
11951,"{'amount_with_offset': '2495000', 'currency': ...",2021 Nissan Qashqai,{'text': '2021 Nissan Qashqui Ti For Sale\nOne...,1.767689e+09,"{'text': 'Melbourne, VIC'}","{'text': 'AU$24,950'}",USED,grey,GASOLINE,black,Nissan,Qashqai,"{'unit': 'KILOMETERS', 'value': 67502}","{'co2_emissions': None, 'engine_size': None, '...",AUTOMATIC
11952,"{'amount_with_offset': '899000', 'currency': '...",2013 Hyundai 2013 Hyundai ix35 LM3 Elite Wagon...,"{'text': ""Will not respond to is this availabl...",1.767573e+09,"{'text': 'Melbourne, VIC'}","{'text': 'AU$8,990'}",USED,grey,PETROL,black,Hyundai,2013 hyundai ix35 lm3 elite wagon 5dr spts aut...,"{'unit': 'KILOMETERS', 'value': 232045}","{'co2_emissions': None, 'engine_size': None, '...",AUTOMATIC
11953,"{'amount_with_offset': '1000', 'currency': 'AU...",2024 Mercedes-Benz GLA250,{'text': 'ALL CAR PARTS FOR SALE \nMelbourne C...,1.741294e+09,"{'text': 'Melbourne, VIC'}",{'text': 'AU$10'},USED,black,PETROL,black,Mercedes-Benz,gla-class,"{'unit': 'KILOMETERS', 'value': 14000}","{'co2_emissions': None, 'engine_size': None, '...",AUTOMATIC


In [409]:
# df["listing_price"].tolist()

In [410]:
# df["formatted_price"].tolist()

In [411]:
df.drop("listing_price",axis=1,inplace=True)

#### Dropping Duplicate Price Column

We dropped `formatted_price` because it contained some incorrect values (especially in the offset) and it was also a duplicate of `listing_price`.  
Since both columns represent the same information, we kept **`listing_price`** as the main price column.

In [412]:
df['creation_time'].describe()

count    1.184100e+04
mean     1.763919e+09
std      6.748094e+06
min      1.641935e+09
25%      1.763257e+09
50%      1.765755e+09
75%      1.767314e+09
max      1.767917e+09
Name: creation_time, dtype: float64

In [413]:
df['creation_year'] = pd.to_datetime(df['creation_time'], unit='s').dt.year
df['creation_month'] = pd.to_datetime(df['creation_time'], unit='s').dt.month
df.drop("creation_time",axis=1,inplace=True)

#### Converting Unix Timestamps to Year and Month

- Our dataset contains a column `creation_time`, which stores **Unix timestamps** — the number of seconds since **January 1, 1970 (UTC)**.  
- Raw timestamps like `1.741294e+09` are **not human-readable** and **cannot be used directly** in machine learning models.  
- So i converted it to year and month
- Dropped creation_time successfully 

In [414]:
df.tail()

,marketplace_listing_title,redacted_description,location_text,formatted_price,condition,vehicle_exterior_color,vehicle_fuel_type,vehicle_interior_color,vehicle_make_display_name,vehicle_model_display_name,vehicle_odometer_data,vehicle_specifications,vehicle_transmission_type,creation_year,creation_month
11950,2002 Mercedes-Benz Sedan,{'text': 'Mercedes Benz kompressor classic sed...,"{'text': 'Adelaide, SA'}","{'text': 'AU$5,900'}",USED,white,GASOLINE,off_white,Mercedes-Benz,sedan,"{'unit': 'KILOMETERS', 'value': 181000}","{'co2_emissions': None, 'engine_size': None, '...",AUTOMATIC,2026.0,1.0
11951,2021 Nissan Qashqai,{'text': '2021 Nissan Qashqui Ti For Sale\nOne...,"{'text': 'Melbourne, VIC'}","{'text': 'AU$24,950'}",USED,grey,GASOLINE,black,Nissan,Qashqai,"{'unit': 'KILOMETERS', 'value': 67502}","{'co2_emissions': None, 'engine_size': None, '...",AUTOMATIC,2026.0,1.0
11952,2013 Hyundai 2013 Hyundai ix35 LM3 Elite Wagon...,"{'text': ""Will not respond to is this availabl...","{'text': 'Melbourne, VIC'}","{'text': 'AU$8,990'}",USED,grey,PETROL,black,Hyundai,2013 hyundai ix35 lm3 elite wagon 5dr spts aut...,"{'unit': 'KILOMETERS', 'value': 232045}","{'co2_emissions': None, 'engine_size': None, '...",AUTOMATIC,2026.0,1.0
11953,2024 Mercedes-Benz GLA250,{'text': 'ALL CAR PARTS FOR SALE \nMelbourne C...,"{'text': 'Melbourne, VIC'}",{'text': 'AU$10'},USED,black,PETROL,black,Mercedes-Benz,gla-class,"{'unit': 'KILOMETERS', 'value': 14000}","{'co2_emissions': None, 'engine_size': None, '...",AUTOMATIC,2025.0,3.0
11954,2015 Mitsubishi Lancer,"{'text': 'Clean, reliable, and fuel-efficient....","{'text': 'Melbourne, VIC'}","{'text': 'AU$9,500'}",USED,white,GASOLINE,black,Mitsubishi,Lancer,"{'unit': 'KILOMETERS', 'value': 252433}","{'co2_emissions': None, 'engine_size': None, '...",AUTOMATIC,2025.0,10.0


## Dictionary Column Extraction

In [415]:
df["dist"] = df["vehicle_odometer_data"].astype(str).str[10:20]
df["dist"].value_counts()

dist
KILOMETERS    11820
                114
one, 'valu       17
MILES', 'v        4
Name: count, dtype: int64

In [416]:
mask = df["dist"] == "KILOMETERS"

df.loc[mask, "distance_in_km"] = (
    df.loc[mask, "vehicle_odometer_data"]
      .astype(str)
      .str.extract(r"'value'\s*:\s*(\d+)", expand=False)
      .astype("Int64")
)
df[["vehicle_odometer_data","distance_in_km"]].tail()

,vehicle_odometer_data,distance_in_km
11950,"{'unit': 'KILOMETERS', 'value': 181000}",181000
11951,"{'unit': 'KILOMETERS', 'value': 67502}",67502
11952,"{'unit': 'KILOMETERS', 'value': 232045}",232045
11953,"{'unit': 'KILOMETERS', 'value': 14000}",14000
11954,"{'unit': 'KILOMETERS', 'value': 252433}",252433


In [417]:
col = ["vehicle_odometer_data","dist"]
df.drop(col,axis=1,inplace=True)

### Extracting Distance in KM from `vehicle_odometer_data`

- The `vehicle_odometer_data` column contained dictionary-like text values such as:

`{'unit': 'KILOMETERS', 'value': 181000}`

### Step 1: Extracting the Unit
We created a new column `dist` to identify the unit (mainly `KILOMETERS` and a few `MILES`) and checked the distribution using `value_counts()`.

### Step 2: Extracting the Odometer Value
For rows where the unit was **KILOMETERS**, we extracted the numeric `value` using regex and stored it in a new column `distance_in_km`.  
We used the `Int64` datatype to safely handle missing values (`NaN`).

### Note on MILES and Noisy Values
A very small number of rows contained the unit as **MILES**, and the remaining non-standard values were extremely few.  
Since their count was negligible compared to the dataset size, we ignored them to keep the data clean and consistent.

### Final Step: Dropping Temporary Columns
After extracting the distance feature, we dropped the columns:
- `vehicle_odometer_data`
- `dist`

In [418]:
df["price_currency"] = df["formatted_price"].astype(str).str[10:13]
df["price_currency"].value_counts()

price_currency
AU$    11782
         114
FRE       19
A$1       16
A$2       10
A$9        3
A$8        3
A$5        3
A$3        3
A$7        2
Name: count, dtype: int64

In [419]:
df["price_In_Dollars"] = (
    df["formatted_price"]
    .astype(str)
    .str.extract(r"(\d[\d,]*)", expand=False) 
    .str.replace(",", "", regex=False)
    .astype("Int64")
)
df[["price_currency","formatted_price"]][df["price_currency"]=="FRE"].head()

,price_currency,formatted_price
413,FRE,{'text': 'FREE'}
595,FRE,{'text': 'FREE'}
742,FRE,{'text': 'FREE'}
1399,FRE,{'text': 'FREE'}
1984,FRE,{'text': 'FREE'}


In [420]:
cols = ["formatted_price","price_currency"]
df.drop(cols,axis=1,inplace=True)

### Cleaning `formatted_price`

The `formatted_price` column had values like `{'text': 'AU$5,900'}`.  
We created `price_currency` to check the currency types and found `AU$`, some `A$1/A$2...` (same currency), and `FRE` (means **FREE**).

Since all `AU$`/`A$` represent Australian Dollars, we ignored currency and extracted only the numeric price value into `price_in_Dollars`.  
`FRE` entries were not used because they have no valid price.

Finally, we dropped the columns:
- `formatted_price`
- `price_currency`


In [421]:
df["location_text"] = df["location_text"].astype(str)
df["city"] = df["location_text"].str.extract(r"'text'\s*:\s*'([^,]+)", expand=False).str.strip()
df["state"] = df["location_text"].str.extract(r",\s*([A-Z]{2,3})'", expand=False).str.strip()

In [422]:
df["state"].value_counts()

state
VIC    2727
QLD    2434
NSW    2305
WA     2217
SA     2151
Name: count, dtype: int64

## Extracting City and State from `location_text`

The `location_text` column contained values like `{'text': 'Melbourne, VIC'}`.  
We extracted two new features from it:

- `city` → city name (e.g., Melbourne)
- `state` → state code (e.g., VIC, NSW, QLD)

After extraction, we used `value_counts()` on `state` to check the distribution across different states.

In [423]:
cols = ["location_text","redacted_description","vehicle_specifications"]
df.drop(cols,axis=1,inplace=True)

## Dropping More Columns

- Dropped `location_text` after extracting `city` and `state`, since it was no longer needed.

- Dropped `redacted_description` because it contains user-written descriptions with inconsistent detail (some rows have long descriptions, while many contain very little information like *"good car"* or *"negotiable"*), making it unreliable for modeling.

- Dropped `vehicle_specifications` because most of its key-value pairs were `null`, and only a small number of rows had useful values like horsepower/engine size, so it was not effective for prediction.


In [424]:
df

,marketplace_listing_title,condition,vehicle_exterior_color,vehicle_fuel_type,vehicle_interior_color,vehicle_make_display_name,vehicle_model_display_name,vehicle_transmission_type,creation_year,creation_month,distance_in_km,price_In_Dollars,city,state
0,2015 Hyundai i30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,<NA>,<NA>,NaN,NaN
1,2020 MG MG3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,<NA>,<NA>,NaN,NaN
2,2016 Hyundai Tucson,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,<NA>,<NA>,NaN,NaN
3,2008 Nissan GT-R,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,<NA>,<NA>,NaN,NaN
4,2014 Toyota 86,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,<NA>,<NA>,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11950,2002 Mercedes-Benz Sedan,USED,white,GASOLINE,off_white,Mercedes-Benz,sedan,AUTOMATIC,2026.0,1.0,181000,5900,Adelaide,SA
11951,2021 Nissan Qashqai,USED,grey,GASOLINE,black,Nissan,Qashqai,AUTOMATIC,2026.0,1.0,67502,24950,Melbourne,VIC
11952,2013 Hyundai 2013 Hyundai ix35 LM3 Elite Wagon...,USED,grey,PETROL,black,Hyundai,2013 hyundai ix35 lm3 elite wagon 5dr spts aut...,AUTOMATIC,2026.0,1.0,232045,8990,Melbourne,VIC
11953,2024 Mercedes-Benz GLA250,USED,black,PETROL,black,Mercedes-Benz,gla-class,AUTOMATIC,2025.0,3.0,14000,10,Melbourne,VIC


In [425]:
df.to_csv("cleaned_car_data.csv", index=False)